In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

In [6]:
bu_feat_df = pd.read_csv('data-test-mle/test_mle/bu_feat.csv.gz', compression='gzip',)

In [7]:
bu_feat_df

,but_num_business_unit,but_postcode,but_latitude,but_longitude,but_region_idr_region,zod_idr_zone_dgr
0,1609,80100,50.096306,1.867220,69,4
1,1995,95700,48.991087,2.520903,3,6
2,1800,73210,45.552933,6.644736,51,4
3,238,47550,44.175370,0.638869,134,10
4,201,13546,43.508418,5.406423,71,10
...,...,...,...,...,...,...
321,52,30900,43.808731,4.367886,6,10
322,1409,83600,43.423641,6.753685,55,10
323,379,31621,43.764840,1.375426,33,3
324,436,14630,49.160428,-0.286385,33,3


In [71]:
train_df = pd.read_csv('data-test-mle/test_mle/train.csv.gz', nrows=10000000, compression='gzip',)
train_df

,day_id,but_num_business_unit,dpt_num_department,turnover
0,2017-09-30,64,127,580.308443
1,2017-09-30,119,127,1512.995918
2,2017-09-30,4,88,668.593556
3,2017-09-30,425,127,0.000000
4,2017-09-30,513,73,0.000000
...,...,...,...,...
277714,2012-12-29,131,73,1.461821
277715,2012-12-29,237,127,759.283046
277716,2012-12-29,129,117,1716.399152
277717,2012-12-29,468,127,1307.357057


In [49]:
train_df[(train_df['day_id']>='2016-01-01') & (train_df['day_id']<'2017-01-01')].groupby('dpt_num_department', as_index=False).turnover.sum()

,dpt_num_department,turnover
0,73,1.974491e+06
1,88,5.696284e+06
2,117,6.535299e+06
3,127,3.378005e+07


In [86]:
train_df

,day_id,but_num_business_unit,dpt_num_department,turnover
0,2017-09-30,64,127,580.308443
1,2017-09-30,119,127,1512.995918
2,2017-09-30,4,88,668.593556
3,2017-09-30,425,127,0.000000
4,2017-09-30,513,73,0.000000
...,...,...,...,...
277714,2012-12-29,131,73,1.461821
277715,2012-12-29,237,127,759.283046
277716,2012-12-29,129,117,1716.399152
277717,2012-12-29,468,127,1307.357057


In [98]:
weekly_turnover_df = train_df.groupby(['day_id', 'dpt_num_department'], as_index=False).turnover.sum()
weekly_turnover_df['day_id'] = pd.to_datetime(weekly_turnover_df['day_id'])  # Ensure day_id is datetime

In [99]:
# Create a new DataFrame with one row per day
expanded_rows = []

for index, row in weekly_turnover_df.iterrows():
    # Create a range of days (7 days) from the day_id
    days = pd.date_range(row['day_id'], periods=7)
    
    # Repeat the department and turnover for each day in the range
    for day in days:
        expanded_rows.append({
            'day_id': day,
            'dpt_num_department': row['dpt_num_department'],
            'turnover': row['turnover']/7
        })

# Convert the list of expanded rows into a DataFrame
daily_turnover_df = pd.DataFrame(expanded_rows)

# Display the expanded dataframe
daily_turnover_df

,day_id,dpt_num_department,turnover
0,2012-12-29,73,1424.297147
1,2012-12-30,73,1424.297147
2,2012-12-31,73,1424.297147
3,2013-01-01,73,1424.297147
4,2013-01-02,73,1424.297147
...,...,...,...
6967,2017-10-02,127,32306.530100
6968,2017-10-03,127,32306.530100
6969,2017-10-04,127,32306.530100
6970,2017-10-05,127,32306.530100


In [100]:
daily_turnover_df['year_month'] = daily_turnover_df['day_id'].dt.to_period('M')

# Group by department and month, then sum the turnover
monthly_turnover = daily_turnover_df.groupby(['year_month', 'dpt_num_department'])['turnover'].sum().reset_index()

In [101]:
monthly_turnover

,year_month,dpt_num_department,turnover
0,2012-12,73,4.272891e+03
1,2012-12,88,3.370403e+04
2,2012-12,117,1.815631e+05
3,2012-12,127,1.064099e+05
4,2013-01,73,5.223764e+04
...,...,...,...
231,2017-09,127,1.289318e+06
232,2017-10,73,1.349337e+04
233,2017-10,88,9.826977e+04
234,2017-10,117,7.712877e+03


In [102]:
monthly_turnover['year_month'] = monthly_turnover['year_month'].astype(str)

In [103]:
fig = px.line(monthly_turnover, 
              x='year_month', 
              y='turnover', 
              color='dpt_num_department', 
              title='Turnover Evolution for Different Business Units',
              markers=True)

# Optional: Customize the layout
fig.update_layout(
    xaxis_title='Day',
    yaxis_title='Turnover',
    legend_title='Business Unit',
    template='plotly',
    width=1200,
    height=800
)

fig.show()

D'après mes connaissances sur décathlon et quelques recherches les sports avec le plus gros turnover chez décathlon sont les sport nautiques, le ski/sports d'hiver, la randonnée, et le fitness. 

Le département 117 fait la majorité de sont turnover entre octobre et mars, on peut supposer que ce département représente les sport d'hiver. 

Le département 127 a le plus gros turnover et a deux pic de ventes, l'été en juillet aout et décembre janvier, cela peut représenter les sports nautiques avec le pic de décembre/janvier qui peut correspondre aux personnes allant en vacances d'hiver au soleil. 

Les deux derniers départements 73 et 88 ont des turnover plus faibles (88 plus important que 73) et sont présents toute l'année en magasin (à la différence du département 117). Les ventes du département 88 sont très stables toute l'année (sauf pic en Novembre 2013) à la différence du département 73 qui a un phénomène de saisonalité sur primtemps et été. Avec ses informations nous pouvons attribuer faire correspondre le département 88 avec le fitness (comportement similaire toute l'année) et et le département 73 avec la randonnée qui est plus généralement pratiquée au printemps et en été même si elle reste présente tout l'année. 

In [106]:
fig = px.line(monthly_turnover[monthly_turnover['dpt_num_department']==88], x='year_month', y='turnover', title='Turnover Evolution Over Time')

fig.update_traces(mode='lines+markers', line=dict(color='red'))
fig.update_layout(xaxis_title='Day', yaxis_title='Turnover')

fig.show()

In [104]:
fig = px.line(monthly_turnover[monthly_turnover['dpt_num_department']==73], x='year_month', y='turnover', title='Turnover Evolution Over Time')

fig.update_traces(mode='lines+markers', line=dict(color='blue'))
fig.update_layout(xaxis_title='Day', yaxis_title='Turnover')

fig.show()

Afin de visualiser plus facilement l'évolution du turnover sur l'année nous avons opté pour un affichage du turover chaque mois plutôt que chaque semaine. Pour cela nous avons déjà regroupé les lignes par semaine et département (la maille business unit ne nous intéresse pas pour répondre aux questions) puis sommes passés d'un dataframe avec une ligne par semaine et département à un dataframe à une ligne par jour et départment avec une valeur de turnover divisée par 7 pour chaque ligne. Puis nous avons regroupé les lignes par mois. Cette méthode est plus précise que de regrouper directement les semaines en mois car des jours risquent de ne pas être attribués aux bons mois. 

In [105]:
fig = px.line(train_df[(train_df['dpt_num_department']==73)].groupby('day_id', as_index=False).turnover.sum(), x='day_id', y='turnover', title='Turnover Evolution Over Time')

fig.update_traces(mode='lines+markers', line=dict(color='blue'))
fig.update_layout(xaxis_title='Day', yaxis_title='Turnover')

fig.show()

In [15]:
test_df = pd.read_csv('data-test-mle/test_mle/test.csv.gz', nrows=10000000, compression='gzip',)
test_df

,day_id,but_num_business_unit,dpt_num_department
0,2017-11-25,95,73
1,2017-11-25,4,117
2,2017-11-25,113,127
3,2017-11-25,93,117
4,2017-11-25,66,127
...,...,...,...
10131,2017-10-07,61,88
10132,2017-10-07,641,117
10133,2017-10-07,724,117
10134,2017-10-07,1302,117
